# Demographic Analysis

In [5]:
# import libraries
import numpy as np
import pandas as pd
import censusdata

In [6]:
# census data downloader function

def get_census_data(tables, state, county, year=2019):

    # Download the data
    data = censusdata.download('acs5', year,  # Use 2019 ACS 5-year estimates
                               censusdata.censusgeo([('state', state), ('county', county), ('tract', '*')]),
                               list(tables.keys()))

    # Rename the column
    data.rename(columns=tables, inplace=True)

    # Extract information from the first column
    data['Name'] = data.index.to_series().apply(lambda x: x.name)
    data['SummaryLevel'] = data.index.to_series().apply(lambda x: x.sumlevel())
    data['State'] = data.index.to_series().apply(lambda x: x.geo[0][1])
    data['County'] = data.index.to_series().apply(lambda x: x.geo[1][1])
    data['Tract'] = data.index.to_series().apply(lambda x: x.geo[2][1])
    data.reset_index(drop=True, inplace=True)
    data = data[['Tract','Name']+list(tables.values())].set_index('Tract')
    return data

def get_census_tract_geom(state_fips, county_fips):

    # find state and county fips here: https://www.census.gov/geographies/reference-files/2017/demo/popest/2017-fips.html
    
    # Download the census tract shapefiles
    tracts = gpd.read_file(f'https://www2.census.gov/geo/tiger/TIGER2019/TRACT/tl_2019_{state_fips}_tract.zip')

    # set index as tract
    tracts = tracts.rename(columns={'TRACTCE':'Tract'}).set_index('Tract')

    # Filter to only King County
    tracts = tracts[tracts['COUNTYFP'] == county_fips]
    tracts = tracts[['geometry']]

    return tracts

In [7]:
# Define the state and county for Seattle
state_fips = '53'  # FIPS code for Washington
county_fips = '033'  # FIPS code for King County

## Interest demographics
* Population
* Sex: Male, and Female 
* Race: White, Black, Asian, and (see distribution for merging)
* Below poverty level
* Median Income
* Median Age
* Median GrossRent
* Median Home Value
* Unemployment Status
* Transportion to work
  - Car
  - Transit
  - Bicycle
  - Walked
  - WFH


In [10]:
tables = {
'B01003_001E': 'TotalPopulation',
'B01001_002E': 'Sex:Male',
'B01001_026E': 'Sex:Female',

'B02001_002E': 'Race:WhiteAlone',
'B02001_003E': 'Race:BlackAlone',
'B02001_004E': 'Race:AmericanIndianAlaskaNativeAlone',
'B02001_005E': 'Race:AsianAlone',
'B02001_006E': 'Race:NativeHawaiianPacificIslanderAlone',
'B02001_007E': 'Race:SomeOtherRaceAlone',
'B02001_008E': 'Race:TwoOrMoreRaces',
'B19013_001E': 'MedianIncome',
'B01002_001E': 'MedianAge',
'B25064_001E': 'MedianGrossRent',
'B25077_001E': 'MedianHomeValue',
'B05001_006E': 'Immigration',
'B23025_005E': 'Umemployment',
'B17001_002E': 'BelowPovertyLevel',
'B08006_002E': 'ToWork:Car',
'B08006_008E': 'ToWork:Transit',
'B08006_014E': 'ToWork:Bike',
'B08006_015E': 'ToWork:Walk',
'B08006_017E': 'ToWork:WFH',

}

In [11]:
census_data = get_census_data(tables, state_fips, county_fips)
census_data

,Name,TotalPopulation,Sex:Male,Sex:Female,BelowPovertyLevel,Race:WhiteAlone,Race:BlackAlone,Race:AmericanIndianAlaskaNativeAlone,Race:AsianAlone,Race:NativeHawaiianPacificIslanderAlone,...,MedianAge,MedianGrossRent,MedianHomeValue,Immigration,ToWork:Car,ToWork:Transit,ToWork:Bike,ToWork:Walk,ToWork:WFH,Umemployment
Tract,,,,,,,,,,,,,,,,,,,,,
020700,"Census Tract 207, King County, Washington",4064,2002,2062,406,2555,434,10,671,20,...,39.4,1617,489000,556,1391,701,0,89,60,92
020800,"Census Tract 208, King County, Washington",4279,1968,2311,217,3618,64,19,423,0,...,48.9,1328,706900,121,1577,296,40,9,237,118
020900,"Census Tract 209, King County, Washington",3420,1827,1593,271,2378,342,5,439,0,...,43.6,1243,590100,358,1208,386,22,76,116,100
021000,"Census Tract 210, King County, Washington",6024,3057,2967,369,3587,455,87,1254,67,...,41.4,1752,477600,364,2508,453,0,65,168,74
022001,"Census Tract 220.01, King County, Washington",5729,2985,2744,236,4107,35,14,473,56,...,43.8,1407,636600,347,2533,122,23,15,178,180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
020100,"Census Tract 201, King County, Washington",3319,1734,1585,90,2681,132,51,280,0,...,52.0,1585,733700,62,1329,114,0,50,147,81
029204,"Census Tract 292.04, King County, Washington",7704,3752,3952,1107,3564,915,40,1336,0,...,27.7,1445,389500,1354,3270,444,0,111,251,223
029305,"Census Tract 293.05, King County, Washington",4177,2190,1987,165,1900,469,0,1338,0,...,40.4,1335,422400,572,1800,142,0,9,85,133
